In [1]:
import pandas as pd
import nltk

In [2]:
import cx_Oracle

def hero_to_df(Userid, Password, Host, Port, SID, query):

    dsn_tns = cx_Oracle.makedsn(host=Host, port=Port, sid=sid_name)
    conn = cx_Oracle.connect(user=Userid, password=Password, dsn=dsn_tns)

    df = pd.read_sql_query(query, conn)
    df.ABSTRACT = df.ABSTRACT.astype(str)
    df.TITLE = df.TITLE.astype(str)
    
    conn.close()
    
    return(df)

## HEROPRD
Userid = "rrice"
Password = "Temp02122020" 
Host = "herodb.rtpnc.epa.gov"
Port = "1521"
sid_name = "heroprd"

def defaultquery(query):
    return hero_to_df(Userid, Password, Host, Port, sid_name, query)

o3litquery = '''select reference_id, year, title, abstract
    from tbl_reference 
    where sdelete = 'No' 
    and reference_id in
    (select reference_id from tbl_reference_usage where usage_id = 19134 and isdeleted = 0)'''

o3includedquery = '''select reference_id, year, title, abstract
    from tbl_reference 
    where sdelete = 'No' 
    and reference_id in
    (select reference_id from tbl_reference_usage where usage_id = 19090 and isdeleted = 0)'''

# check if reference is included in first draft ISA
df = defaultquery(o3litquery)
df2 = defaultquery(o3includedquery)

df['included'] = df.REFERENCE_ID.isin(df2.REFERENCE_ID)
df.head()

#2755: Lead ISA 2013 Included
#19134: Ozone ISA Lit search included
#19090: Ozone ISA Included in first draft

,REFERENCE_ID,YEAR,TITLE,ABSTRACT,included
0,3399492,2015.0,Function of ABA in Stomatal Defense against Bi...,None,False
1,3399493,2015.0,Reactive oxygen species in cell wall metabolis...,Although reactive oxygen species (ROS) are hig...,False
2,3399494,2015.0,Mycorrhiza-induced lower oxidative burst is re...,Mechanisms of arbuscular mycorrhiza (AM)-induc...,False
3,4305685,2016.0,Osmotic and hydraulic adjustment of mangrove s...,Salinity tolerance in plant species varies wid...,False
4,4332492,2017.0,Cross-talk between high light stress and plant...,Little is known about how plants deal with art...,False


In [3]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

stop_words = set(stopwords.words('english'))
ps = PorterStemmer()

def nltk_process(words):

    tokens = word_tokenize(words)
    
    stemmed_words = []
    for word in tokens:
        stemmed_words.append(ps.stem(word))
    
    stopword_filtered = [w for w in stemmed_words if w not in stop_words]
    
    #remove digits and punctuation   
    pattern = re.compile(r'[a-zA-Z]+')
    cleaned = [x for x in stopword_filtered if pattern.search(x)]
    
    return cleaned
    
nltkdf = pd.DataFrame()
nltkdf['Text'] = df['TITLE']+df['ABSTRACT'].dropna()  ###########changed to abstract
nltkdf['Text_Processed'] = nltkdf['Text'].apply(lambda x: nltk_process(x))

In [4]:
nltkdf.join(df).iloc[:,:].to_pickle('nltk_stemmed.pkl')

In [5]:
nltkdf['Text_Processed']

0        [function, aba, stomat, defens, biotic, drough...
1        [reactiv, oxygen, speci, cell, wall, metabol, ...
2        [mycorrhiza-induc, lower, oxid, burst, relat, ...
3        [osmot, hydraul, adjust, mangrov, sapl, extrem...
4        [cross-talk, high, light, stress, plant, defen...
                               ...                        
31901    [multiyear, applic, wrf/chem, continent, US, m...
31902    [invers, structur, winter, ozon, distribut, ui...
31903    [develop, applic, next, gener, air, sensor, ne...
31904    [applic, chemic, dispers, model, dure, high, o...
31905    [continu, emiss, carbon, tetrachlorid, unit, s...
Name: Text_Processed, Length: 31906, dtype: object